In [1]:
# !pip install tqdm

In [2]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm
tqdm.pandas()

import os
from pprint import pprint
import re
from collections import defaultdict

In [3]:
DATA_ADDRESS = "./data"
os.listdir(DATA_ADDRESS)

['.gitkeep',
 'correlation_df_mean_no_clean.csv',
 'correlation_df_std_no_clean.csv',
 'df_5s.pkl',
 'df_5s_source.pkl',
 'df_index_test.pkl',
 'df_index_train.pkl',
 'df_merged.pkl',
 'df_merged_pivot.xlsx',
 'df_processed_descriptive_statistics.pickle',
 'df_processed_descriptive_statistics.pkl',
 'df_processed_descriptive_statistics_mel40_mfcc20.pkl',
 'df_processed_simple.pkl',
 'df_processed_stat_no_trim_cleaned_fft2048_mel128_mfcc20.pkl',
 'df_processed_stat_no_trim_cleaned_fft2048_mel40_mfcc17.pkl',
 'df_processed_stat_no_trim_cleaned_fft512_mel64_mfcc17.pkl',
 'df_processed_stat_no_trim_no_clean_fft2048_mel128_mfcc20.pkl',
 'df_processed_stat_no_trim_no_clean_fft512_mel128_mfcc17.pkl',
 'preprocessed',
 'spearman_correlation_df_mean_cleaned.csv',
 'spearman_correlation_df_mean_no_clean.csv',
 'spearman_correlation_df_std_cleaned.csv',
 'spearman_correlation_df_std_no_clean.csv',
 'VOiCES_devkit']

In [4]:
TRAIN_DIR = os.path.join(DATA_ADDRESS,'preprocessed','train')
os.listdir(TRAIN_DIR)

['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12500-12799.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6499.pkl',
 'mel_len15_fft2048_mels1

In [5]:
TEST_DIR = os.path.join(DATA_ADDRESS,'preprocessed','test')
os.listdir(TEST_DIR)

['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6399.pkl',
 'mel_len5_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len5_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len5_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mel_len5_fft2048_mels128_mfcc17_12000

## Load data

In [6]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj


def load_dataset(dataset_dir,train_files):
    ls_X = []
    for k,v in tqdm(train_files.items()):  
        train_file = v['file_name']
        file_dir = os.path.join(dataset_dir, train_file)
        ls_X.append(load_pickle(file_dir))
    return np.concatenate(ls_X, axis=0)

def load_to_dataframe(X,y):
    X_df = pd.DataFrame(X)

    # Convert the 1D array y to a pandas Series
    y_series = pd.Series(y, name='target')

    # Concatenate the features and target into a single DataFrame
    data = pd.concat([X_df, y_series], axis=1)

    return data

### loading y

In [7]:
df_raw = pd.read_pickle(os.path.join(DATA_ADDRESS,'df_5s_source.pkl'))

In [8]:
df_raw[df_raw['category'] == 'train']

,origin_folder,speaker,category,source
0,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
1,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
2,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
3,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
4,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...
...,...,...,...,...
2314,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...
2315,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...
2316,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...
2317,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...


In [9]:
y_train = np.array(df_raw[df_raw['category']=='train']['speaker']).astype('float32')
print(y_train.shape)
y_train

(1552,)


array([4967., 4967., 4967., ..., 1066., 1066., 1066.], dtype=float32)

In [10]:
y_test = np.array(df_raw[df_raw['category']=='test']['speaker']).astype('float32')
print(y_test.shape)
y_test

(775,)


array([2137., 2137., 2137., 2137., 2137., 2137., 2137., 2137., 2628.,
       2628., 2628., 2628., 2628., 2628., 2628., 2628., 1603., 1603.,
       1603., 1603., 1603., 1603., 1603., 1603., 1226., 1226., 1226.,
       1226., 1226., 1226., 1226., 5139., 5139., 5139., 5139., 5139.,
       5139., 5139., 5139., 7437., 7437., 7437., 7437., 7437., 7437.,
       7437.,  176.,  176.,  176.,  176.,  176.,  176.,  176.,  176.,
        118.,  118.,  118.,  118.,  118.,  118.,  118.,  118., 3214.,
       3214., 3214., 3214., 3214., 3214., 3214., 3214., 3816., 3816.,
       3816., 3816., 3816., 3816., 3816., 3816., 1263., 1263., 1263.,
       1263., 1263., 1263., 1263., 1263., 5731., 5731., 5731., 5731.,
       5731., 5731., 5731., 5731.,  192.,  192.,  192.,  192.,  192.,
        192.,  192., 7594., 7594., 7594., 7594., 7594., 7594., 7594.,
       7594.,  175.,  175.,  175.,  175.,  175.,  175.,  175.,  175.,
       3389., 3389., 3389., 3389., 3389., 3389., 3389., 5322., 5322.,
       5322., 5322.,

### X_train

In [11]:

file_prefix = 'source_mfcc_len5_fft2048_mels128_mfcc17_'

# List all files in the directory
pprint(os.listdir(TRAIN_DIR))




['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12500-12799.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6499.pkl',
 'mel_len15_fft2048_mels1

In [12]:
# Filter files that start with the specified prefix
files = os.listdir(TRAIN_DIR)
train_files_mfcc = [file for file in files if file.startswith(file_prefix)]
pprint(train_files_mfcc)

['source_mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
 'source_mfcc_len5_fft2048_mels128_mfcc17_1000-1499.pkl',
 'source_mfcc_len5_fft2048_mels128_mfcc17_1500-1551.pkl',
 'source_mfcc_len5_fft2048_mels128_mfcc17_500-999.pkl']


In [13]:
# Filter files that start with the specified prefix
files = os.listdir(TEST_DIR)
test_files_mfcc = [file for file in files if file.startswith(file_prefix)]
pprint(test_files_mfcc)

['source_mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
 'source_mfcc_len5_fft2048_mels128_mfcc17_500-774.pkl']


In [14]:
# Regular expression to match two numbers at the end of the file name
pattern = re.compile(r'_(\d+)-(\d+)\.pkl$')
n_interval = 500

# Extract the numbers
dir_files = defaultdict(dict)
for file_name in train_files_mfcc:
    match = pattern.search(file_name)
    n1 = int(match.group(1))
    n2 = int(match.group(2))
    file_num = n1/n_interval
    dir_files[file_num] = {
        'begin':n1,
        'end':n2,
        'file_name' : file_name,
        'metadata':df_raw[df_raw['category'] == 'train'].iloc[n1:n2+1,:]
    }
sorted_dir_files_train = {k: dir_files[k] for k in sorted(dir_files)}
pprint(sorted_dir_files_train)

{0.0: {'begin': 0,
       'end': 499,
       'file_name': 'source_mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
       'metadata':                origin_folder speaker category  \
0    source-16k/train/sp4967    4967    train   
1    source-16k/train/sp4967    4967    train   
2    source-16k/train/sp4967    4967    train   
3    source-16k/train/sp4967    4967    train   
4    source-16k/train/sp4967    4967    train   
..                       ...     ...      ...   
782  source-16k/train/sp4586    4586    train   
783  source-16k/train/sp4586    4586    train   
784  source-16k/train/sp4586    4586    train   
785  source-16k/train/sp4586    4586    train   
794  source-16k/train/sp2294    2294    train   

                                                source  
0    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...  
1    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...  
2    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...  
3    5seconds-16k-source/train/sp4967/Lab41-SRI-VOi..

In [15]:
# Regular expression to match two numbers at the end of the file name
pattern = re.compile(r'_(\d+)-(\d+)\.pkl$')
n_interval = 500

# Extract the numbers
dir_files = defaultdict(dict)
for file_name in test_files_mfcc:
    match = pattern.search(file_name)
    n1 = int(match.group(1))
    n2 = int(match.group(2))
    file_num = n1/n_interval
    dir_files[file_num] = {
        'begin':n1,
        'end':n2,
        'file_name' : file_name,
        'metadata':df_raw[df_raw['category'] == 'test'].iloc[n1:n2+1,:]
    }
sorted_dir_files_test = {k: dir_files[k] for k in sorted(dir_files)}
pprint(sorted_dir_files_test)

{0.0: {'begin': 0,
       'end': 499,
       'file_name': 'source_mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
       'metadata':                origin_folder speaker category  \
8     source-16k/test/sp2137    2137     test   
9     source-16k/test/sp2137    2137     test   
10    source-16k/test/sp2137    2137     test   
11    source-16k/test/sp2137    2137     test   
12    source-16k/test/sp2137    2137     test   
...                      ...     ...      ...   
1435  source-16k/test/sp0166    0166     test   
1436  source-16k/test/sp0166    0166     test   
1437  source-16k/test/sp0166    0166     test   
1438  source-16k/test/sp0166    0166     test   
1455  source-16k/test/sp8419    8419     test   

                                                 source  
8     5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...  
9     5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...  
10    5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...  
11    5seconds-16k-source/test/sp2137/Lab41-SRI-V

In [16]:
X_train = load_dataset(TRAIN_DIR,sorted_dir_files_train)
X_train.shape

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 117.85it/s]


(1552, 17, 216)

In [17]:
X_test= load_dataset(TEST_DIR,sorted_dir_files_test)
X_test.shape

100%|██████████| 2/2 [00:00<00:00, 117.86it/s]


(775, 17, 216)

In [18]:
df_train = load_to_dataframe(X_train.reshape(X_train.shape[0], -1),y_train)
print(df_train.shape)
# Check the first few rows of the DataFrame
print(df_train.head())

(1552, 3673)
           0          1          2          3          4          5  \
0 -80.199921 -78.522278 -78.041252 -78.678085 -79.135452 -79.036118   
1 -54.440174 -55.169106 -55.757004 -56.125408 -56.542290 -56.143124   
2 -57.445320 -55.021637 -55.360378 -59.159683 -65.186165 -54.589661   
3 -36.961510 -35.436325 -37.526371 -36.496399 -41.023388 -51.365696   
4 -77.139420 -77.644806 -78.030930 -78.258194 -75.489655 -73.090729   

           6          7          8          9  ...      3663      3664  \
0 -79.023300 -78.891022 -78.589790 -78.287941  ...  0.463551  0.172204   
1 -54.699898 -53.520699 -52.859917 -53.442547  ... -1.133153 -0.999535   
2 -52.513134 -55.607136 -57.488655 -59.600983  ...  0.008717 -0.693536   
3 -53.648964 -53.890499 -52.856178 -52.297760  ...  0.023778  0.023779   
4 -75.297333 -75.117676 -73.656136 -71.496193  ...  0.210924 -0.147249   

       3665      3666      3667      3668      3669      3670      3671  \
0 -0.357568 -0.051849  0.275233 -0.04379

In [19]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,3663,3664,3665,3666,3667,3668,3669,3670,3671,target
0,-80.199921,-78.522278,-78.041252,-78.678085,-79.135452,-79.036118,-79.023300,-78.891022,-78.589790,-78.287941,...,0.463551,0.172204,-0.357568,-0.051849,0.275233,-0.043794,-0.308624,-0.698927,-0.431659,4967.0
1,-54.440174,-55.169106,-55.757004,-56.125408,-56.542290,-56.143124,-54.699898,-53.520699,-52.859917,-53.442547,...,-1.133153,-0.999535,-0.878778,-0.591550,-0.171469,0.112264,0.066884,0.534677,1.084320,4967.0
2,-57.445320,-55.021637,-55.360378,-59.159683,-65.186165,-54.589661,-52.513134,-55.607136,-57.488655,-59.600983,...,0.008717,-0.693536,-0.626800,0.141316,-0.014385,0.379013,0.452272,0.804542,1.672258,4967.0
3,-36.961510,-35.436325,-37.526371,-36.496399,-41.023388,-51.365696,-53.648964,-53.890499,-52.856178,-52.297760,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,4967.0
4,-77.139420,-77.644806,-78.030930,-78.258194,-75.489655,-73.090729,-75.297333,-75.117676,-73.656136,-71.496193,...,0.210924,-0.147249,-0.700613,-1.409750,-1.428489,-0.651570,-0.245459,0.205517,0.488121,4967.0


In [20]:
df_test = load_to_dataframe(X_test.reshape(X_test.shape[0], -1),y_test)
print(df_test.shape)
# Check the first few rows of the DataFrame
print(df_test.head())

(775, 3673)
           0          1          2          3          4          5  \
0 -81.884140 -80.688713 -80.817665 -81.237801 -79.507782 -67.654274   
1 -47.831593 -46.041744 -48.758507 -54.395081 -51.878159 -51.375824   
2 -61.573219 -58.873405 -61.184879 -62.705585 -59.836666 -55.406189   
3 -68.330528 -59.480625 -54.732224 -54.440231 -58.174480 -59.918407   
4 -81.452156 -80.327385 -79.937065 -79.419128 -79.475914 -79.748436   

           6          7          8          9  ...      3663      3664  \
0 -56.935516 -53.059200 -53.265228 -55.379414  ... -0.266179  0.126821   
1 -51.875473 -54.703400 -62.250816 -71.048164  ... -0.440574 -0.498703   
2 -52.880264 -52.793549 -55.413013 -59.198479  ... -0.537550 -0.386306   
3 -64.322296 -66.132416 -64.338234 -65.220566  ...  0.023778  0.023779   
4 -79.301659 -79.082138 -78.951134 -78.804688  ...  1.448353  0.783824   

       3665      3666      3667      3668      3669      3670      3671  \
0 -0.255052 -0.809486 -1.071832 -0.980961

In [21]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,3663,3664,3665,3666,3667,3668,3669,3670,3671,target
0,-81.884140,-80.688713,-80.817665,-81.237801,-79.507782,-67.654274,-56.935516,-53.059200,-53.265228,-55.379414,...,-0.266179,0.126821,-0.255052,-0.809486,-1.071832,-0.980961,-1.091645,-0.136575,-0.270308,2137.0
1,-47.831593,-46.041744,-48.758507,-54.395081,-51.878159,-51.375824,-51.875473,-54.703400,-62.250816,-71.048164,...,-0.440574,-0.498703,-0.457408,-0.690353,-0.552562,-0.699915,-0.689881,-0.692636,-0.751200,2137.0
2,-61.573219,-58.873405,-61.184879,-62.705585,-59.836666,-55.406189,-52.880264,-52.793549,-55.413013,-59.198479,...,-0.537550,-0.386306,-0.377339,-0.284840,0.246314,0.134439,-0.350945,0.064106,0.156389,2137.0
3,-68.330528,-59.480625,-54.732224,-54.440231,-58.174480,-59.918407,-64.322296,-66.132416,-64.338234,-65.220566,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,2137.0
4,-81.452156,-80.327385,-79.937065,-79.419128,-79.475914,-79.748436,-79.301659,-79.082138,-78.951134,-78.804688,...,1.448353,0.783824,0.244643,-0.293547,-0.417796,-1.162916,-0.878246,-1.066555,-0.691707,2137.0


In [22]:
def prepare_df(df,file_dir):
    df['pkl_file_dir'] = file_dir
    return df.reset_index(names='old_index').reset_index(names='file_index')

In [23]:
for k,v in tqdm(sorted_dir_files_train.items()):
    file_dir = os.path.join('preprocessed','train',v['file_name']).replace("\\", "/")
    v['metadata'] = prepare_df(v['metadata'],file_dir)

100%|██████████| 4/4 [00:00<00:00, 571.31it/s]


In [24]:
for k,v in tqdm(sorted_dir_files_test.items()):
    file_dir = os.path.join('preprocessed','test',v['file_name']).replace("\\", "/")
    v['metadata'] = prepare_df(v['metadata'],file_dir)

100%|██████████| 2/2 [00:00<00:00, 998.41it/s]


In [26]:
sorted_dir_files_test[1]['metadata']

,file_index,old_index,origin_folder,speaker,category,source,pkl_file_dir
0,0,1456,source-16k/test/sp8419,8419,test,5seconds-16k-source/test/sp8419/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
1,1,1457,source-16k/test/sp8419,8419,test,5seconds-16k-source/test/sp8419/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
2,2,1458,source-16k/test/sp8419,8419,test,5seconds-16k-source/test/sp8419/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
3,3,1459,source-16k/test/sp8419,8419,test,5seconds-16k-source/test/sp8419/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
4,4,1460,source-16k/test/sp8419,8419,test,5seconds-16k-source/test/sp8419/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
...,...,...,...,...,...,...,...
270,270,2322,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
271,271,2323,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
272,272,2324,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
273,273,2325,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...


In [27]:
df_list = []
for k,v in tqdm(sorted_dir_files_train.items()):
    df_list.append(v['metadata'])

df_index_train = pd.concat(df_list).reset_index(drop=True)
df_index_train.to_pickle(os.path.join(DATA_ADDRESS,'df_index_source_train.pkl'))


100%|██████████| 4/4 [00:00<?, ?it/s]


In [28]:
df_index_train

,file_index,old_index,origin_folder,speaker,category,source,pkl_file_dir
0,0,0,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
1,1,1,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
2,2,2,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
3,3,3,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
4,4,4,source-16k/train/sp4967,4967,train,5seconds-16k-source/train/sp4967/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
...,...,...,...,...,...,...,...
1547,47,2314,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
1548,48,2315,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
1549,49,2316,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...
1550,50,2317,source-16k/train/sp1066,1066,train,5seconds-16k-source/train/sp1066/Lab41-SRI-VOi...,preprocessed/train/source_mfcc_len5_fft2048_me...


In [29]:
df_list = []
for k,v in tqdm(sorted_dir_files_test.items()):
    df_list.append(v['metadata'])

df_index_test = pd.concat(df_list).reset_index(drop=True)
df_index_test.to_pickle(os.path.join(DATA_ADDRESS,'df_index_source_test.pkl'))

100%|██████████| 2/2 [00:00<?, ?it/s]


In [30]:
df_index_test

,file_index,old_index,origin_folder,speaker,category,source,pkl_file_dir
0,0,8,source-16k/test/sp2137,2137,test,5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
1,1,9,source-16k/test/sp2137,2137,test,5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
2,2,10,source-16k/test/sp2137,2137,test,5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
3,3,11,source-16k/test/sp2137,2137,test,5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
4,4,12,source-16k/test/sp2137,2137,test,5seconds-16k-source/test/sp2137/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
...,...,...,...,...,...,...,...
770,270,2322,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
771,271,2323,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
772,272,2324,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
773,273,2325,source-16k/test/sp2785,2785,test,5seconds-16k-source/test/sp2785/Lab41-SRI-VOiC...,preprocessed/test/source_mfcc_len5_fft2048_mel...
